In [ ]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image


In [ ]:
def preprocess_image(image):
    # Resize image to (256, 256) using OpenCV
    resized_image = cv2.resize(image, (256, 256))
    # Normalize pixel values to range [0, 1]
    normalized_image = resized_image / 255.0
    return normalized_image

In [ ]:
def preprocess_dataset(dataset):
    preprocessed_images = []
    labels = []

    for image_batch, label_batch in dataset:
        for image, label in zip(image_batch, label_batch):
            preprocessed_images.append(preprocess_image(image.numpy()))
            labels.append(label.numpy())

    return np.array(preprocessed_images), np.array(labels)


In [ ]:
breeds = ["beagle", "bernese_mountain_dog", "doberman", "labrador_retriever", "siberian_husky"]
args = {
    'labels': 'inferred',
    'label_mode': 'categorical',
    'batch_size': 32,
    'image_size': (256,256),
    'seed':1,
    'validation_split':0.2,
    'class_names':breeds
}

# Load dataset using TensorFlow
train_dataset = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/dog_images/images",
    subset="training",
    **args
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/dog_images/images",
    subset="validation",
    **args
)

In [ ]:
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def create_model(input_shape, num_classes):
    network = [
        layers.Conv2D(16, 4, padding='same', activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(),
        layers.Dropout(.2),
        layers.Conv2D(32, 4, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(.2),
        layers.Conv2D(64, 4, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(.2),
        layers.Conv2D(128, 4, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes)
    ]
    model = Sequential(network)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, train_data, test_data, epochs=5):
    X_train, y_train = train_data
    X_test, y_test = test_data
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs)
    history_df = pd.DataFrame.from_dict(history.history)
    return history_df, model

In [ ]:
# Preprocess train and test datasets
train_data = preprocess_dataset(train_dataset)
test_data = preprocess_dataset(test_dataset)

# Create and compile the model
model = create_model(input_shape=(256, 256, 3), num_classes=len(breeds))

# Train the model
history_df, trained_model = train_model(model, train_data, test_data, epochs=15)

history_df[['accuracy', 'val_accuracy']].plot()